In [1]:
!git clone https://github.com/nkt780426/3d_face_recognition_magface.git

Cloning into '3d_face_recognition_magface'...
remote: Enumerating objects: 611, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 611 (delta 46), reused 63 (delta 21), pack-reused 516 (from 1)
Receiving objects: 100% (611/611), 98.30 MiB | 43.13 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
!mv 3d_face_recognition_magface/going_modular .

In [3]:
!gdown --quiet --folder https://drive.google.com/drive/folders/1-1KWxxoID1SrKzs1BYvEj6whqBzz1LtW?usp=sharing

In [4]:
import warnings
warnings.filterwarnings("ignore")
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import torch

from going_modular.dataloader.multitask import create_concatv3_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.model.ConcatMTLFaceRecognition import ConcatMTLFaceRecognitionV3
from going_modular.loss.ConcatMultiTaskLoss import ConcatMultiTaskLoss
from going_modular.train_eval.concat_train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    'type': 'concat3',
    
    # Thư mục
    'dataset_dir': '/kaggle/input/hoangvn-3dmultitask',
    'checkpoint_dir': '/kaggle/working/checkpoint/multi/',
    'checkpoint_1': '/kaggle/working/multi/normalmap/checkpoint.pth',
    'checkpoint_2': '/kaggle/working/multi/albedo/checkpoint.pth',
    'checkpoint_3': '/kaggle/working/multi/depthmap/checkpoint.pth',
        
    # Cấu hình train
    'device': device,
    'epochs': 119,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'base_lr': 1e-4,
    
    # Cấu hình network
    'backbone': 'miresnet18',
    'embedding_size': 512,
    'num_classes': None,
    'loss_gender_weight': 30,
    'loss_emotion_weight': 5,
    'loss_pose_weight': 30,
    'loss_spectacles_weight': 10,
    'loss_facial_hair_weight': 10,
}

CONFIGURATION['num_classes'] = len(os.listdir('/kaggle/input/hoangvn-3dmultitask/Albedo/train'))

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})


test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
], additional_targets={0.9420

train_dataloader, test_dataloader = create_concatv3_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)
checkpoint_1 = torch.load(CONFIGURATION['checkpoint_1'])
mtl_normalmap = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
mtl_normalmap.load_state_dict(checkpoint_1['model_state_dict'])

checkpoint_2 = torch.load(CONFIGURATION['checkpoint_2'])
mtl_albedo = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
mtl_albedo.load_state_dict(checkpoint_2['model_state_dict'])

checkpoint_3 = torch.load(CONFIGURATION['checkpoint_3'])
mtl_depthmap = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
mtl_depthmap.load_state_dict(checkpoint_3['model_state_dict'])

model = ConcatMTLFaceRecognitionV3(mtl_normalmap, mtl_albedo, mtl_depthmap, CONFIGURATION['num_classes'])

for param in model.mtl_normalmap.parameters():
    param.requires_grad = False
    
for param in model.mtl_albedo.parameters():
    param.requires_grad = False

for param in model.mtl_depthmap.parameters():
    param.requires_grad = False

criterion = ConcatMultiTaskLoss(os.path.join(CONFIGURATION['dataset_dir'], 'train_set.csv'), CONFIGURATION)
optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
# Khởi tạo scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=1000,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=0
)      
checkpoint_path = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

In [5]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler, 
    early_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 1:
╒══════════════════╤═════════════╤══════════════════════╕
│ Metric           │       Train │ Test                 │
╞══════════════════╪═════════════╪══════════════════════╡
│ loss             │ 26.5253     │ -                    │
├──────────────────┼─────────────┼──────────────────────┤
│ loss_id          │ 23.5472     │ -                    │
├──────────────────┼─────────────┼──────────────────────┤
│ loss_gender      │  0.0293193  │ 0.0233631438575685   │
├──────────────────┼─────────────┼──────────────────────┤
│ loss_emotion     │  0.069323   │ 0.5259189117777472   │
├──────────────────┼─────────────┼──────────────────────┤
│ loss_pose        │  0.00811135 │ 0.013303831320566435 │
├──────────────────┼─────────────┼──────────────────────┤
│ loss_facial_hair │  0.0802793  │ 0.11087920119365056  │
├──────────────────┼─────────────┼──────────────────────┤
│ loss_spectacles  │  0.0705848  │ 0.07084887754172087  │
├──────────────────┼─────────────┼──────────────────────┤
│ auc

In [6]:
!zip -r output.zip checkpoint/multi/

  adding: checkpoint/multi/ (stored 0%)
  adding: checkpoint/multi/concat3/ (stored 0%)
  adding: checkpoint/multi/concat3/models/ (stored 0%)
  adding: checkpoint/multi/concat3/models/best_cosine_auc_26.pth (deflated 7%)
  adding: checkpoint/multi/concat3/models/best_euclidean_auc_91.pth (deflated 7%)
  adding: checkpoint/multi/concat3/models/checkpoint.pth (deflated 7%)
  adding: checkpoint/multi/concat3/logs/ (stored 0%)
  adding: checkpoint/multi/concat3/logs/AUC_pose_test/ (stored 0%)
  adding: checkpoint/multi/concat3/logs/AUC_pose_test/events.out.tfevents.1736982532.315934d839e9.18.22 (deflated 69%)
  adding: checkpoint/multi/concat3/logs/Loss_emotion_train/ (stored 0%)
  adding: checkpoint/multi/concat3/logs/Loss_emotion_train/events.out.tfevents.1736982532.315934d839e9.18.9 (deflated 63%)
  adding: checkpoint/multi/concat3/logs/Loss_pose_test/ (stored 0%)
  adding: checkpoint/multi/concat3/logs/Loss_pose_test/events.out.tfevents.1736982532.315934d839e9.18.8 (deflated 61%)
  ad